calling the previous notebook here!

In [0]:
%run /Workspace/Users/dhivyadharshini298@gmail.com/databricks-code-repository/Bread_and_Butter_Transformation_Analytical_operations_DSL/4.DATA_CURATION_OR_PRE_WRANGLING

#DATA_WRANGLING TECHNIQUES


##1. JOINS


In [0]:
custdf1=formatted_derived_df
df1=custdf1.where("custid in (4000100,4000101,4000108)")
df2=custdf1.where("custid in (4000100,4000102,4000103)")
df1=df1.withColumn("custid",when((col("custid")==4000108),lit(None)).otherwise(col("custid")))
df2=custdf1.where("custid in (4000100,4000102,4000103,4000104)")
df2=df2.withColumn("custid",when((col("custid")==4000104),lit(None)).otherwise(col("custid")))
display(df1)
display(df2)


In [0]:
print("inner")
display(df1.join(df2,how='inner',on='custid'))#1
print("semi")
display(df1.join(df2,how='semi',on='custid'))#1
print("left")
display(df1.join(df2,how='left',on='custid'))#2
print("anti")
display(df1.join(df2,how='anti',on='custid'))#2
print("right")
display(df1.join(df2,how='right',on='custid'))#3
print("full")
display(df1.join(df2,how='full',on='custid'))#4

- Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last year, so i can send them some offers.
- Solution: First Lets try to bring transaction dataset to perform joins

In [0]:
from pyspark.sql.functions import *
strt1="txnid long,txndt string,custid long,amt float,cat string,prod string,city string,state string,spendby string"
txnsdf=spark.read.schema(strt1).csv("/Volumes/workspace/default/volumewe47_datalake/txns_2025.txt",header=False)
print(len(txnsdf.collect()))
txnsdf_munged=txnsdf.distinct().dropDuplicates(["txnid"]).na.drop()
print(len(txnsdf_munged.collect()))
txnsdf_munged.printSchema()

In [0]:
txnsdf2=txnsdf_munged.withColumn("txndt",to_date("txndt",'MM-dd-yyyy'))
txnsdf2.printSchema()
display(txnsdf2.summary())
display(txnsdf2.take(20))

- Requirement: I need to do analytics/reporting of top 3 customers who did more and highest sum amount of transactions in our business last year, so i can send them some offers.
- First - Let us perform join to achieve the expected requirement result (one to many join)
- Second - Do grouping & aggregation
- Third - Sorting
- Fourth - limiting

In [0]:
custdf1=formatted_derived_df
custdf1.printSchema()

In [0]:
joineddf=custdf1.join(other=txnsdf2,on="custid",how="inner")#Preferable join is "inner" (whether you mention or not)
print(joineddf.count())

In [0]:
joineddf.groupBy("custid","fullname").count().show()
joineddf.groupBy("custid","fullname").count().orderBy(desc("count")).show(3)

In [0]:
top3=joineddf.groupBy("custid","fullname","prof").agg(sum("amt").alias("tot_amt"))\
.withColumn("custtype",lit("top3"))\
.orderBy("tot_amt",ascending=False)\
.limit(3) #Top 3 customers purchased for highest amount last year
top3.display()

In [0]:
least3=joineddf.groupBy("custid","fullname","prof").agg(sum("amt").alias("tot_amt")).withColumn("custtype",lit("least3")).orderBy("tot_amt",ascending=True).limit(3)#Top 3 customers purchased for lowest amount last year
display(top3.union(least3))

- Requirement: I need to do analytics/reporting of the dormant/**in active**  customer who didn't visited our business last year
- custdf1 dataframe contains all customer info of the complete years of my business
- txnsdf2 dataframe contains last 1 year transactions

In [0]:
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left")#Returns all data from left df and only matching data from right df rest of data will be having nulls
display(custdidnotvisitedlastyeardf.where("txnid is null"))#inactive customers

- Requirement: I need to do analytics/reporting of the total number of **active** customer who visited our business last year
- custdf1 dataframe contains all customer info of the complete years of my business
- txnsdf2 dataframe contains last 1 year transactions

In [0]:
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left")#Returns all data from left df and only matching data from right df rest of data will be having nulls
print(custdidnotvisitedlastyeardf.where("txnid is not null").selectExpr("custid").distinct().count())#active customers of last 1 year
display(custdidnotvisitedlastyeardf.limit(10))

- how many active customers of 2025 december

In [0]:
txnsdfdec2025=txnsdf2.where("month(txndt)=12")
custdidnotvisiteddecmonthdf=custdf1.join(txnsdfdec2025,on="custid",how="left")
print("inactive customers of december month",custdidnotvisiteddecmonthdf.where("txnid is null").selectExpr("custid").distinct().count())
print("active customers of december month",custdidnotvisiteddecmonthdf.where("txnid is not null").selectExpr("custid").distinct().count())

In [0]:
#I want achieve right outer join without using right keyword?
custdidnotvisitedlastyeardf=txnsdf2.join(custdf1,on="custid",how="left")#this is right join now
     

In [0]:
#how many products are transacted in december month regardless of customer information
custdidnotvisiteddecmonthdf=custdf1.join(txnsdfdec2025,on="custid",how="right")
display(custdidnotvisiteddecmonthdf)
print("Overall number of products ",txnsdfdec2025.select("prod").distinct().count())
print("Products had no purchases by the inactive customers",custdidnotvisiteddecmonthdf.where("age is null").selectExpr("prod").distinct().count())#active products of december month


In [0]:
#how many products are transacted in december month (show me both customer and products regardless of customer information or product information matching)
custdidnotvisiteddecmonthdf=custdf1.join(txnsdf2,on="custid",how="full")
print(custdidnotvisiteddecmonthdf.count())

In [0]:
print("Overall number of products ",txnsdfdec2025.select("prod").distinct().count())
print("Total Products available ",custdidnotvisiteddecmonthdf.selectExpr("prod").distinct().count())#active products of december month
print("Products had purchases by customers",custdidnotvisiteddecmonthdf.where("age is not null").selectExpr("prod").distinct().count())#active products of december month
print("Products not purchased by customers",custdidnotvisiteddecmonthdf.where("age is null").selectExpr("prod").distinct().count())#active products of december month
print("Total customer available, but no transaction made ",custdidnotvisiteddecmonthdf.where("txnid is null").selectExpr("fullname").distinct().count())#active products of december month
print("Total customer available, transaction made ",custdidnotvisiteddecmonthdf.where("txnid is not null").selectExpr("fullname").distinct().count())#active products of december month

##2. LOOKUPS

- Lookup and enrichment is the process of looking up for some data attributes using the key to identify the presence of the values (not the actual values are returned) lookup? existence check that returns boolean 
- Eg. whether this particular customer made a transaction or not

**semi/anti (best), left/inner (least bother)**

In [0]:
#I need to produce only the df1 data present in df2 (by doing lookup)
df1=custdf1.where("custid in (4000100,4000101)")
df1.show()
df2=custdf1.where("custid in (4000100,4000102,4000103)")
#With the left customer info provided, check whether it is present in the right df and display both left and right matching data
display(df1.alias("a").join(df2.alias("b"),how='inner',on='custid').select(col("a.*")))#Returns only the matching data from both left & right df also, comparing with right, hence Inner join will lookup and enrich also

#With the left customer info provided, check whether it is present in the right df and display only left matching data (right)
display(df1.join(df2,how='left_semi',on='custid'))#Returns only the matching data from the left df, comparing with right - hence semi is a lookup join

In [0]:
#Requirement: I need to do analytics/reporting of the total number of active customer who visited our business last year
#custdf1 dataframe contains all customer info of the complete years of my business
#txnsdf2 dataframe contains last 1 year transactions
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left_semi")#Returns all data from left df only that matches with the right df
print("Total active customers of last year ",custdidnotvisitedlastyeardf.count())
#display(custdidnotvisitedlastyeardf.where("txnid is not null").selectExpr("custid").distinct().count())#inactive customers of last 1 year
     

In [0]:
#Requirement: I need to do analytics/reporting of the total number of IN active customer who didn't visited our business last year
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="leftanti")#Returns all data from left df only that DID NOT matches with the right df
display(custdidnotvisitedlastyeardf)
print("Total IN active customers of last year ",custdidnotvisitedlastyeardf.count())

##3.LOOKUP & ENRICHMENT

 Lookup and enrichment is the process of looking up for for some data attributes using the key and enrich the values. Eg. Checking the bank account balance for a given acct id. lookup & enrichment? existence check that returns result/value that can be used for enriching our data

**Lookup and enrichment using joins - left_join (best), inner, right(least bother)**

In [0]:
lookup_enriched_df=custdf1.join(txnsdf2,how='left',on='custid')
display(lookup_enriched_df.where("txnid is not null"))
display(lookup_enriched_df.where("txnid is null"))

In [0]:
lookup_enriched_df=custdf1.join(txnsdf2,how='inner',on='custid')
#display(lookup_enriched_df.where("txnid is not null"))#Only the matching data between left and right alone returned
#display(lookup_enriched_df.where("txnid is null"))#no data returned
display(lookup_enriched_df)

##4.SCHEMA MODELING(DENORMALIZATION)

Use inner(mostly),left/right/full(we can use depends on the business) We can build Datawarehouse components (dimension, fact tables) appling joins on the tables to achieve different types of schemas

1. Star Schema
2. Snowflake Schema
3. Starflake Schema

In [0]:
#Implementing only star schema model (fact surrounded by one or more main dimension) to produce denormalized (single dataset) fact table
dimensiondf=custdf1
#display(dimensiondf)
factdf=txnsdf2
#display(factdf)

In [0]:
final_denormalized_df1=dimensiondf.join(factdf,how='inner',on='custid')#applying inner to reject/miss data from both side
final_denormalized_df2=dimensiondf.join(factdf,how='left',on='custid')#applying left outer to ensure all customer data is present
final_denormalized_df3=dimensiondf.join(factdf,how='right',on='custid')#applying right outer to ensure all transaction data is present
final_denormalized_df4=dimensiondf.join(factdf,how='full',on='custid')#applying full outer to not reject/miss any data
display(final_denormalized_df1)#Final fact table to load in our lakehouse(delta table)

##5.WINDOWING

row_number, rank, dense_rank
Application of Windowing & Analytical functions
Most frequent interview questions asked...

1. how do you create surrogate/seq number in tables?
2. Top N analysis of data?
3. how do you remove duplicates in the table data?
select * from (select *,row_number() over(partition by department_id order by salary) rownum from employees)temp where rownum=1;

In [0]:
#Creation of Surrogate/Sequence Numbers
#1. how do you create surrogate/seq number in tables? 
# If the source given some key (natural key) and we wanted to maintain our own key in our environment then we create surrogate key (unique key with sequence) Eg. For a bank - Aadharnumber(naturalkey) & bankacctnumber(surrogatekey)
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1
dfcusts_surrogate_seqnum_df=dfcusts.select(row_number().over(Window.orderBy("custid","amt")).alias("primary_sk"),"*")
#Performance degradation: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
#or 

In [0]:
#or 
#One more efficient way to create surrogate key without sorting the data (Interview question)
dfcusts_surrogate_seqnum_df=dfcusts.select(monotonically_increasing_id().alias("primary_sk"),"*")
display(dfcusts_surrogate_seqnum_df)

In [0]:
#2. Top N Analysis (Very very important for interview..)
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1.select("custid","fullname","prof","txnid","amt","prof").where("custid in (4000001,4000002,4000003)")
display(dfcusts)

In [0]:
print("1.Row number")
custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(desc("amt"))))
display(custtrans_topn_df)

In [0]:
custtrans_topn_df.where("custrno=1").show()#Show me the only top 1 transaction made by the every customer
custtrans_topn_df.where("custrno=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custrno=5").show()#Fifth Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custrno<=3").show()#Show me the top 3 transactions made by the customers (with the same values of transaction)

In [0]:
custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)
custtrans_topn_df.where("custrno=1").show()#Least transaction made by each customer
custtrans_topn_df.where("custrno=2").show()#Second Least transaction made by each customer

In [0]:
print("Rank & Dense_rank usage:")
custtrans_topn_df=dfcusts.withColumn("custrnk",rank().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custdrnk",dense_rank().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)


In [0]:
print("Rank")
#Rank returns multiple rows per customer if the top transaction is same value
custtrans_topn_df.where("custrnk=1").show()#Show me the only top 1 transaction made by the every customer 
custtrans_topn_df.where("custrnk=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custrnk=4").show()#Four Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custrnk=5").show()#this record is missing (because skipped by rank function)
custtrans_topn_df.where("custrnk=6").show()#this record is missing (because skipped by rank function)


In [0]:
print("Dense Rank")
#Dense Rank returns multiple rows per customer if the top transaction is same value and the next value of the rank is dense/not skipped
custtrans_topn_df.where("custdrnk=1").show()#Show me the only top 1 transaction made by the every customer 
custtrans_topn_df.where("custdrnk=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custdrnk=4").show()#Fifth Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custdrnk=5").show()#this record is available (because dense rank function is used)
custtrans_topn_df.where("custdrnk=6").show()#this record is available (because dense rank function is used)


In [0]:
print("Rownumber, Rank & Dense_rank usage:")
custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custrnk",rank().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custdrnk",dense_rank().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)
     

##6.ANALYTICAL

Functions used for performing some analytical operations on the data in multiple dimensions. We can use windowing or grouping or grouping & aggregation functions to perform analytics/analytical functionalities

1. Hierarchical Analytics (lead & lag)
2. Aggregation analytics (grouping & aggregations)
3. Multi dimensional analytics (rollup, cube, pivot)

###1. Hierarchical Analytics (lead & lag)

In [0]:
#Let us perform hierarchical analytics by using lead and lag functions
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1.select("custid","fullname","prof","txnid","amt","prof","txndt").where("custid in (4000001,4000002,4000003)")
#Lead help us arrive the next value
hierarchicalleaddf=dfcusts.withColumn("nextamt",lead("amt").over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleaddf)
#Lag help us arrive the prior value
hierarchicallagdf=dfcusts.withColumn("prioramt",lag("amt").over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicallagdf)

print("application of lead and lag")
print("How to control the hierarchy (for example I want to compare immediate next transaction or 2 levels of transactions")
#syntax lead(col,offset,default) or lag(col,offset,default)
hierarchicalleaddf=dfcusts.withColumn("nextamt",lead("amt",2,-1).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleaddf)
#Lead help us arrive the prior value
hierarchicallagdf=dfcusts.withColumn("prioramt",lag("amt",2,0).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicallagdf)

print("Requirement: I want to analyse the purchase pattern/trend of my customer")
hierarchicalleadlagdf=dfcusts.withColumn("nextamt",lead("amt",1,0).over(Window.partitionBy("custid").orderBy("txndt")))\
.withColumn("prioramt",lag("amt",1,0).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleadlagdf)
#trendanalysisdf=hierarchicalleadlagdf.withColumn("trend",when((col("nextamt")>col("amt")),"up").when((col("nextamt")

###2. Aggregation analytics (grouping & aggregations)

In [0]:
dfcusts=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000001 and 4000500")
dfcusts.show(2)
groupedaggdf=dfcusts.groupBy("prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage"))
display(groupedaggdf)
groupedaggdf=dfcusts.groupBy("state","city","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage"))
display(groupedaggdf)

###3. Multi dimensional analytics (rollup, cube, pivot)

In [0]:
#Multi dimensional analytics (rollup, cube, pivot)
print("rollup produces the aggregated result in rolled UP dimensions")
groupedaggdf=dfcusts.rollup("state","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","prof")
display(groupedaggdf)

print("cube produces the aggregated result in all dimensions (rollup and rolldown)")
groupedaggdf=dfcusts.cube("state","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","prof")
display(groupedaggdf)

In [0]:
#Pivot operation (costly function)
print("pivot produces the aggregated result of the dimensions in a columnar fashion")
groupedaggdf=dfcusts.groupBy("state","city","spendby").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","city","spendby")
display(groupedaggdf)
groupedaggdf=dfcusts.groupBy("state","city").pivot("spendby").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","city")
display(groupedaggdf)

##7.SET OPERATIONS

In [0]:
#Set operation (union, unionAll, insetsect, minus/except/subract)
dfcusts1=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000001 and 4000010")
dfcusts2=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000005 and 4000015")
unionalldf=dfcusts1.unionAll(dfcusts2)
#display(unionalldf)#returns all data from both the dataframes with duplicates
print(unionalldf.count())
uniondf=dfcusts1.union(dfcusts2).distinct()
display(uniondf)#returns all data from both the dataframes with duplicates  (it will not work like this in spark), we have to use distinct explicitly
print(uniondf.count())

#intersection
commondf=dfcusts1.intersect(dfcusts2)
display(commondf)
print(commondf.count())

#subtract
diffdf=dfcusts1.subtract(dfcusts2)#returns df1-df2 data (excessive df1 data)
display(diffdf)
print(diffdf.count())